# Import Libraries

In [6]:
import tensorflow as tf
from tensorflow import keras
from matplotlib import cm as c

from utils.val import *
from engine.train import *

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


tf.enable_eager_execution() 

# Define/Import your own model

In [1]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input, Dropout, Concatenate

Using TensorFlow backend.
/Users/yeojin/anaconda3/envs/yjenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/yeojin/anaconda3/envs/yjenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/yeojin/anaconda3/envs/yjenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/yeojin/anacon

In [2]:
#################### edit this part ######################
#use Keras Model instead of Sequential
#in case you need to import MCNN model in another file

def MCNN(input_shape=None):
    inputs=Input(shape=input_shape)
    
    # column 1
    #column_1 = Conv2D(filters, kernel_size, padding, activation, name)(inputs)
    column_1 = Conv2D(16, (9, 9), padding='same', activation='relu', name='col1_conv1')(inputs)
    column_1 = MaxPooling2D(2)(column_1)
    column_1 = Conv2D(32, (7, 7), padding='same', activation='relu', name='col1_conv2')(column_1)
    column_1 = MaxPooling2D(2)(column_1)
    column_1 = Conv2D(16, (7, 7), padding='same', activation='relu', name='col1_conv3')(column_1)
    column_1 = Conv2D(8, (7, 7), padding='same', activation='relu', name='col1_conv4')(column_1)

    # column 2
    column_2 = Conv2D(20, (7, 7), padding='same', activation='relu', name='col2_conv1')(inputs)
    column_2 = MaxPooling2D(2)(column_2)
    column_2 = Conv2D(40, (5, 5), padding='same', activation='relu', name='col2_conv2')(column_2)
    column_2 = MaxPooling2D(2)(column_2)
    column_2 = Conv2D(20, (5, 5), padding='same', activation='relu', name='col2_conv3')(column_2)
    column_2 = Conv2D(10, (5, 5), padding='same', activation='relu', name='col2_conv4')(column_2)

    # column 3
    column_3 = Conv2D(24, (5, 5), padding='same', activation='relu', name='col3_conv1')(inputs)
    column_3 = MaxPooling2D(2)(column_3)
    column_3 = Conv2D(48, (3, 3), padding='same', activation='relu', name='col3_conv2')(column_3)
    column_3 = MaxPooling2D(2)(column_3)
    column_3 = Conv2D(24, (3, 3), padding='same', activation='relu', name='col3_conv3')(column_3)
    column_3 = Conv2D(12, (3, 3), padding='same', activation='relu', name='col3_conv4')(column_3)

    # merge feature map of 3 columns in last dimension
    merges = Concatenate(axis=-1)([column_1, column_2, column_3])
    
    # density map
    density_map = Conv2D(1, (1, 1), padding='same', activation='linear', name='density_conv')(merges)

    model = Model(inputs=inputs, outputs=density_map)
    return model
    

In [3]:
input_shape=(None,None,1)
model=MCNN(input_shape)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 1 0                                            
__________________________________________________________________________________________________
col1_conv1 (Conv2D)             (None, None, None, 1 1312        input_1[0][0]                    
__________________________________________________________________________________________________
col2_conv1 (Conv2D)             (None, None, None, 2 1000        input_1[0][0]                    
__________________________________________________________________________________________________
col3_conv1 (Conv2D)             (None, None, None, 2 624         input_1[0][0]                    
____________________

# Train model

In [5]:
# define epochs and learning rates 
#############edit this if you want to change epochs or lr###############
fit(model, epochs=1, learning_rate = 0.02)

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
Learning started. It takes sometime.
Epoch: 1 loss = 160.82553101 Test MAE =  160.7493
Learning Finished!


# Visualize test outputs

In [ ]:
#############change test_input to check how your model works with a test data#######
generate_images(model, test_input = 'ShanghaiTech/part_A/test_data/images/IMG_100.jpg')